## Imports

In [1]:
import os
import nibabel as nib
import glob

### Reading all BraTS21 examples, change the root_path to you base address

In [2]:
root_path = 'Sample BraTS21 50 Examples'
data_list = sorted(glob.glob(root_path + '/*'))        #list of paths of the inside files

#### The following function is responsible for returning the indices of the depth of the volume that contains labels.

In [3]:
def find_tumor(wid, hei):
    slice2D = img.get_fdata()[:, :, i]
    for j in range(wid):
        for k in range(hei):
            if slice2D[j, k] != 0:
                return i
    return 0

#### Creating results folder that will contain the cropped volumes

In [4]:
# Create results folder
results_path_path = r'Sample BraTS21 50 Examples Depth Cropped'
if not os.path.exists(results_path_path):
    os.makedirs(results_path_path)

#### Looping in every example and in every module in the volume to crop it to contain only the part that is already segmented

In [8]:
for vol in data_list:
    modules_list = sorted(glob.glob(vol + '/*'))          #modules list of a single volume
    t1_vol_path = modules_list[2]                        #path of T1 volume is always the third ('sorted')
    img = nib.load(t1_vol_path)                          #segmentation volume

    height, width, depth = img.shape
    filled_slices = []                                    #slices with at least one label

    for i in range(depth):
        depth_idx = find_tumor(width, height)
        if depth_idx != 0:
            filled_slices.append(depth_idx)

    min_depth_idx, max_depth_idx = filled_slices[0], filled_slices[-1]
    for module_path in modules_list:
        module_vol = nib.load(module_path)
        cropped_vol = module_vol.get_fdata()[:, :, min_depth_idx : (max_depth_idx+1)]
        nifti_img =  nib.Nifti1Image(cropped_vol, module_vol.affine)     # to save this 3D (ndarry) numpy use this

        vol_new_path = results_path_path + '/' +vol.split('\\')[1]
        if not os.path.exists(vol_new_path):
            os.makedirs(vol_new_path)

        module_new_path = vol_new_path + '/' + module_path.split('\\')[-1]
        nib.save(nifti_img, module_new_path)

#### Testing Random image

In [9]:
test_path = r'C:\Users\Mohamed Bushnaq\PycharmProjects\Slices\Sample BraTS21 50 Examples Depth Cropped\BraTS2021_00020\BraTS2021_00020_t1.nii.gz'
img_test = nib.load(test_path)

img_test.shape

(240, 240, 137)

#### Image shape was (240, 240, 155) Before cropping, now it's (240, 240, 137) for this example (Random)